# Drowsiness Prediction    

In [1]:
from scipy.spatial import distance
from imutils import face_utils
import numpy as np
import winsound
import imutils
import dlib
import cv2
import time

In [2]:
def loadModel():
    from tensorflow.keras.models import load_model
    model = load_model("model\CNN_Model_For_Drowsyness_Detection.h5")
    return model

In [3]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [4]:
def init():
    
    model = loadModel()
    
    ## Face coordinates from dat file 
    detect = dlib.get_frontal_face_detector()
    predict = dlib.shape_predictor("ShapePredictor\shape_predictor_68_face_landmarks.dat")# Dat file is the crux of the code
    
    return model , detect , predict
    

In [5]:
def endCameraFeed(cap):
    cv2.destroyAllWindows()
    cap.release()
    #print(released)
    

In [6]:
def main():
    
    cap=cv2.VideoCapture(0)
    flag=0
    playingAlarm = time.time()
    
    ## initial parameters
    thresh_model = 0.50
    thresh_EAR = 0.25
    frame_check = 10
    
    ## eye coordinates 
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]
    
    ## intializing 
    model , detect , predict = init()
    
    
    ## main loop
    while True:
        ret, frame=cap.read()
        frame = imutils.resize(frame, width=600)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        subjects = detect(gray, 0)
        for subject in subjects:
            shape = predict(gray, subject)
            shape = face_utils.shape_to_np(shape)#converting to NumPy Array


        ## getting the left Eye Image from the Photo
            (x1,y1) , (x2,y2) = (shape[43][0] , shape[44][1] ), (shape[46][0] , shape[47][1]) 
            leftEye_Image = cv2.resize(frame[y1-15:y2+15 , x1-15:x2+15] , (86,86))
            leftEye_Image = np.reshape(leftEye_Image, (1,86,86,3))
            #print(leftEye_Image)

        ## getting the right Eye Image from the Photo
            (x1,y1) , (x2,y2) = (shape[37][0] , shape[38][1] ), (shape[40][0] , shape[41][1]) 
            rightEye_Image = cv2.resize(frame[y1-15:y2+15 , x1-15:x2+15] , (86,86))
            rightEye_Image = np.reshape(rightEye_Image, (1,86,86,3))
            #print(leftEye_Image)

        ## calculating EAR for both the Eyes:
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)


        ## Model Prediction
            leftEye_prediction = model.predict(leftEye_Image)
            rightEye_prediction = model.predict(rightEye_Image)


            l1 = (leftEAR + rightEAR)/2
            l2 = (leftEye_prediction[0][1] + rightEye_prediction[0][1])/2


            if l1 < thresh_EAR or l2 <= thresh_model:
                flag += 1

                if flag >= frame_check:
                
                    ## winsound.PlaySound("alert.wav", True)
                    if(time.time() > playingAlarm+1.7):
                        playingAlarm = time.time()
                        winsound.PlaySound("alert.wav", True)
                
                    '''
                    cv2.putText(frame, "****************ALERT!****************", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    '''
                    cv2.putText(frame, "Drowsiness Detected.... Playing Alarm...!!", (10,425),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                    #print ("Drowsy")
            else:
                flag = 0

            # print ("flag:",flag)
        if len(subjects) == 0:
            flag = 0
            cv2.putText(frame, "Face Not Found...!!", (10,425),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)


        cv2.putText(frame, "Sleeping Frames : "+str(flag) , (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)    

        cv2.imshow("Frame", frame)

        
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            endCameraFeed(cap)
            break

    

In [7]:
main()